## Import Packages

In [1]:
import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random
import json

from PIL import Image, ImageOps, ImageEnhance
import numbers

import torchattacks
from torchattacks import CW

from tensorboardX import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline

## Normalize Dataset

In [2]:
#Creating dataset using torch dataloaders
batch_size_train = 128
batch_size_test = 1000
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))]) # Normalizing dataset

# Training dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=transform),
  batch_size=batch_size_train, shuffle=True)

# Test dataset
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=transform),
  batch_size=batch_size_test, shuffle=True)

# Initialize GPU
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


## Initial Model Structure

In [3]:
# Initial model structure
class HNet(nn.Module):    
    def __init__(self):
        super(HNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x     

## SubModel Structure

In [4]:
# Submodel Structure for training residual
class NHNet(nn.Module):    
    def __init__(self):
        super(NHNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x   

## Initialize Data Structures

In [5]:
# Initialize some data structures to store useful data
train_losses = []
train_counter = []
test_losses = []

## Initialize Parameters, optimizer and loss function

In [6]:
# Create the initial model
initial_model = HNet()

#Create the optimizer for the initial model
optimizer = optim.Adam(initial_model.parameters(), lr=0.003)

# Create Loss function for the intial model
criterion = nn.CrossEntropyLoss()

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Change model into cuda mode
if torch.cuda.is_available():
    initial_model = initial_model.cuda()
    criterion = criterion.cuda()

## Training Function for Initial Model

In [7]:
# Initial Model Training Function
def train(epoch):
    initial_model.train()
#     exp_lr_scheduler.step()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = initial_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        # torch.save(initial_model.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/model.pth')
        # torch.save(optimizer.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/optimizer.pth')
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

## Evaluation Function for Intial Model

In [8]:
#Initial Model Evaluating Function
def evaluate(data_loader):
    initial_model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
        
            output = initial_model(data)
        
            loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    test_losses.append(loss)    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

## Start Training the Initial Model

In [17]:
# Start Training
n_epochs = 65

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.602848
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.549241
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.581888
Train Epoch: 0 [51200/60000 (85%)]	Loss: 1.536988


C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  if __name__ == '__main__':



Average loss: 1.5576, Accuracy: 54379/60000 (90.632%)

Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.562668
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.532746
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.530872
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.574295

Average loss: 1.5313, Accuracy: 55880/60000 (93.133%)

Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.515534
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.524624
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.525929
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.502936

Average loss: 1.5157, Accuracy: 56773/60000 (94.622%)

Train Epoch: 3 [12800/60000 (21%)]	Loss: 1.501767
Train Epoch: 3 [25600/60000 (43%)]	Loss: 1.551804
Train Epoch: 3 [38400/60000 (64%)]	Loss: 1.538258
Train Epoch: 3 [51200/60000 (85%)]	Loss: 1.534766

Average loss: 1.5180, Accuracy: 56624/60000 (94.373%)

Train Epoch: 4 [12800/60000 (21%)]	Loss: 1.514447
Train Epoch: 4 [25600/60000 (43%)]	Loss: 1.528231
Train Epoch: 4 [38400/60000 (64%)]	Loss: 1.516948
Train Epoch: 4 [51200/6000

Train Epoch: 32 [51200/60000 (85%)]	Loss: 1.508222

Average loss: 1.5023, Accuracy: 57525/60000 (95.875%)

Train Epoch: 33 [12800/60000 (21%)]	Loss: 1.508074
Train Epoch: 33 [25600/60000 (43%)]	Loss: 1.547074
Train Epoch: 33 [38400/60000 (64%)]	Loss: 1.548809
Train Epoch: 33 [51200/60000 (85%)]	Loss: 1.521500

Average loss: 1.5059, Accuracy: 57314/60000 (95.523%)

Train Epoch: 34 [12800/60000 (21%)]	Loss: 1.508026
Train Epoch: 34 [25600/60000 (43%)]	Loss: 1.484142
Train Epoch: 34 [38400/60000 (64%)]	Loss: 1.492401
Train Epoch: 34 [51200/60000 (85%)]	Loss: 1.523629

Average loss: 1.5080, Accuracy: 57187/60000 (95.312%)

Train Epoch: 35 [12800/60000 (21%)]	Loss: 1.507460
Train Epoch: 35 [25600/60000 (43%)]	Loss: 1.491980
Train Epoch: 35 [38400/60000 (64%)]	Loss: 1.514934
Train Epoch: 35 [51200/60000 (85%)]	Loss: 1.492401

Average loss: 1.5127, Accuracy: 56892/60000 (94.820%)

Train Epoch: 36 [12800/60000 (21%)]	Loss: 1.508026
Train Epoch: 36 [25600/60000 (43%)]	Loss: 1.484588
Train Epoch

Train Epoch: 64 [25600/60000 (43%)]	Loss: 1.500212
Train Epoch: 64 [38400/60000 (64%)]	Loss: 1.523651
Train Epoch: 64 [51200/60000 (85%)]	Loss: 1.518947

Average loss: 1.5082, Accuracy: 57166/60000 (95.277%)



## Save or Load the Initial Model

In [18]:
# Save this model so that I dont have to train again in the future
torch.save(initial_model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')

In [28]:
# Load the model from local file
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

HNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

## Calculatee mse residual and normalize the result to values between 0 ~ 1

In [8]:
# Function used to calculate mse residual based on the wiki
def mseresidual(y, F):
    residual = y - F
    absolute = torch.abs(residual)
    residual = residual / torch.max(absolute)
    return residual

## Actual Code for gradient Boosting

In [15]:
#Main Function used to train and find optimal gamma for submodels
#input: intial model that's already trained
#M is number of submodels needed to be trained
def GradientBoosting(initial_model, M):
    gamma_exp = torch.ones([M], dtype = torch.float64) # used to hold the final optimized gamma
    models = [] # used to hold all the models
    residual_list = [] # used to hold the residual of each batch calculated
    for m in range(M):
        # Intialize submodels
        Hmodel = NHNet()
        Hcriterion = nn.MSELoss()
        if torch.cuda.is_available():
            Hmodel = Hmodel.cuda()
            gamma_exp = gamma_exp.cuda()
            Hcriterion = Hcriterion.cuda()
            
        # Start Training
        epoch = 40
        Hoptimizer = optim.Adam(Hmodel.parameters(), lr=0.001)
        for i in range(epoch):
            Hmodel.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                # Create one-hot label target tensor
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                # Calculate F(x)
                output = initial_model(data)
                # Calculate the output from all the models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        output = output.cuda()
                        model = model.cuda()
                    output = output + gamma_exp[j] * model(data)
#                 print("output is:", output)
                #Convert into Onehot label so that it would be able to calculate the residual
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                #Calculate Residual
#                 print("target_onehot is:", target_onehot)
#                 print("output is:", output)
                residual = mseresidual(target_onehot, output)
                houtput = Hmodel(data)
#                 print("houtput is:", houtput)
                residual = residual.type(torch.cuda.FloatTensor)
                houtput = houtput.type(torch.cuda.FloatTensor)
                #Calculate the loss
                loss = Hcriterion(houtput, residual)
                Hoptimizer.zero_grad()
                loss.backward(retain_graph = True)
                Hoptimizer.step()
                # Print out current Process
                if (batch_idx + 1)% 100 == 0 and i % 10 == 0:
                    print('Train Epoch: Model Number: {} {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            m+1,i, (batch_idx + 1) * len(data), len(train_loader.dataset),
                            100. * (batch_idx + 1) / len(train_loader), loss.item()))
        models.append(Hmodel)
        
        # Initialize a random gamma
        gamma = torch.rand(1, requires_grad=True, device="cuda")
        Goptimizer = optim.Adam([gamma], lr=0.01)
        Gcriterion = nn.MSELoss()
        # Start finding the best gamma
        for i in range(20):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                    Hmodel = Hmodel.cuda()
                    gamma = gamma.cuda()
                    
                #Calculate the initial output
                Goptimizer.zero_grad()  
                output = initial_model(data)
                #Calculate the final output by combining all previous models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        model = model.cuda()
                        output = output.cuda()
                        gamma_temp = gamma_exp[j]
                        gamma_temp = gamma_temp.cuda()
                    output = output + gamma_temp * model(data)
                # Covert into one-hot label
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                # Get the currect model output
                temp = Hmodel(data)
                # Find the current ensemble model output
                predicted = output + gamma * temp
                # Calculate the loss
                loss = Gcriterion(predicted, target_onehot)
                loss.backward(retain_graph = True)
                # Optimize the gamma
                Goptimizer.step()  
        gamma_exp[m] = gamma
    print(gamma_exp)
    return models, gamma_exp

## Adaptive Diversity Promoting Training

In [11]:
log_offset = 1e-20
det_offset = 1e-6

In [26]:
def ADPGradient(initial_model, M):
    gamma_exp_adp = torch.ones([M], dtype = torch.float64)
    models_adp = [] # used to hold all the models
    residual_list = [] # used to hold the residual of each batch calculated
    epoch = 30
    writer = SummaryWriter()
    for m in range(M):
        # Intialize submodels
        Hmodel = NHNet()
        Hcriterion = nn.MSELoss()
        if torch.cuda.is_available():
            Hmodel = Hmodel.cuda()
            gamma_exp_adp = gamma_exp_adp.cuda()
            Hcriterion = Hcriterion.cuda()
            params = []
            params += list(Hmodel.parameters())
            for j in models_adp:
                params += list(j.parameters())
            Hoptimizer = optim.Adam(params, lr=0.001, weight_decay=1e-4, eps=1e-7)
        for i in range(epoch):
            Hmodel.train()
            for j in models_adp:
                j.train()
                
            losses = 0
            ce_losses = 0
            ee_losses = 0
            det_losses = 0
            alpha = 2.0
            beta = 0.5
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                y_true = torch.zeros(data.size(0), nb_digits).cuda()
                y_true.scatter_(1, target.view(-1,1), 1)
                ce_loss = 0
                mask_non_y_pred = []
                ensemble_probs = 0
                for k in range(m+1):
                    output = initial_model(data)
                    target = target.view(-1,1)
                    target_onehot.zero_()
                    target_onehot.scatter_(1, target, 1)
                    for j in range(m):
                        model = models_adp[j]
                        if torch.cuda.is_available():
                            model = model.cuda()
                            output = output.cuda()
                            gamma_temp = gamma_exp_adp[j]
                            gamma_temp = gamma_temp.cuda()
                        if k < m and j == k:
                            residual = mseresidual(target_onehot, output)
                            houtput = model(data)
                            residual = residual.type(torch.cuda.FloatTensor)
                            houtput = houtput.type(torch.cuda.FloatTensor)
                            ce_loss += Hcriterion(houtput, residual)
                            y_pred = F.softmax(output, dim=-1)
#                             y_pred = torch.reshape(output,dim=-1)
                            bool_R_y_true = torch.eq(torch.ones_like(y_true) - y_true, torch.ones_like(y_true))
                            mask_non_y_pred.append(torch.masked_select(y_pred, bool_R_y_true).reshape(-1, nb_digits-1))
                            ensemble_probs += y_pred
                        output = output + gamma_temp * model(data)
                    
                    if k == m:
                        residual = mseresidual(target_onehot, output)
                        houtput = Hmodel(data)
                        residual = residual.type(torch.cuda.FloatTensor)
                        houtput = houtput.type(torch.cuda.FloatTensor)
                        ce_loss += Hcriterion(houtput, residual)
                        y_pred = F.softmax(output, dim=-1)
#                         y_pred = torch.reshape(output,dim=-1)
                        bool_R_y_true = torch.eq(torch.ones_like(y_true) - y_true, torch.ones_like(y_true))
                        mask_non_y_pred.append(torch.masked_select(y_pred, bool_R_y_true).reshape(-1, nb_digits-1))
                        ensemble_probs += y_pred

                ensemble_probs = ensemble_probs / (m+1)
                ensemble_entropy = torch.sum(-torch.mul(ensemble_probs, torch.log(ensemble_probs + log_offset)), dim=-1).mean()

                mask_non_y_pred = torch.stack(mask_non_y_pred, dim=1)

#                 print("mask_non_y_pred shape is:", mask_non_y_pred.shape)
                assert mask_non_y_pred.shape == (data.size(0), m+1, nb_digits-1)
                mask_non_y_pred = mask_non_y_pred / torch.norm(mask_non_y_pred, p=2, dim=-1, keepdim=True)
                matrix = torch.matmul(mask_non_y_pred, mask_non_y_pred.permute(0, 2, 1))
                log_det = torch.logdet(matrix+det_offset*torch.eye((m+1), device=matrix.device).unsqueeze(0)).mean()

                loss = ce_loss - alpha * ensemble_entropy - beta * log_det

                losses += loss.item()
                ce_losses += ce_loss.item()
                ee_losses += ensemble_entropy.item()
                det_losses += -log_det.item()

                Hoptimizer.zero_grad()
                loss.backward(retain_graph = True)
                Hoptimizer.step()
                if (batch_idx + 1) % 400 == 0:
                    print('Train Epoch: Model Number: {} {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            m+1,i, (batch_idx + 1) * len(data), len(train_loader.dataset),
                            100. * (batch_idx + 1) / len(train_loader), loss.item()))
                if m == 0:
                    writer.add_scalar('train/ce_loss_0', ce_losses/len(train_loader), i)
                    writer.add_scalar('train/ee_loss_0', ee_losses/len(train_loader), i)
                    writer.add_scalar('train/det_loss_0', det_losses/len(train_loader), i)
                    writer.add_scalar('train/log_det_0', log_det, i)
                if m == 1:
                    writer.add_scalar('train/ce_loss_1', ce_losses/len(train_loader), i)
                    writer.add_scalar('train/ee_loss_1', ee_losses/len(train_loader), i)
                    writer.add_scalar('train/det_loss_1', det_losses/len(train_loader), i)
                    writer.add_scalar('train/log_det_1', log_det, i)
                if m == 2:
                    writer.add_scalar('train/ce_loss_2', ce_losses/len(train_loader), i)
                    writer.add_scalar('train/ee_loss_2', ee_losses/len(train_loader), i)
                    writer.add_scalar('train/det_loss_2', det_losses/len(train_loader), i)
                    writer.add_scalar('train/log_det_2', log_det, i)
        models_adp.append(Hmodel)
        
            # Initialize a random gamma
        gamma = torch.rand(1, requires_grad=True, device="cuda")
        Goptimizer = optim.Adam([gamma], lr=0.01)
        Gcriterion = nn.MSELoss()
        # Start finding the best gamma
        for i in range(20):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                    Hmodel = Hmodel.cuda()
                    gamma = gamma.cuda()
                    
                #Calculate the initial output
                Goptimizer.zero_grad()  
                output = initial_model(data)
                #Calculate the final output by combining all previous models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        model = model.cuda()
                        output = output.cuda()
                        gamma_temp = gamma_exp_adp[j]
                        gamma_temp = gamma_temp.cuda()
                    output = output + gamma_temp * model(data)
                # Covert into one-hot label
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                # Get the currect model output
                temp = Hmodel(data)
                # Find the current ensemble model output
                predicted = output + gamma * temp
                # Calculate the loss
                loss = Gcriterion(predicted, target_onehot)
                loss.backward(retain_graph = True)
                # Optimize the gamma
                Goptimizer.step()  
        gamma_exp_adp[m] = gamma
    print(gamma_exp_adp)
    writer.close()
    return models_adp, gamma_exp_adp

## Start Gradient Boosting Training

In [16]:
num_of_models = 3
models, gamma_exp = GradientBoosting(initial_model, num_of_models)

Train Epoch: Model Number: 1 0 [12800/60000 (21%)]	Loss: 0.021916
Train Epoch: Model Number: 1 0 [25600/60000 (43%)]	Loss: 0.023078
Train Epoch: Model Number: 1 0 [38400/60000 (64%)]	Loss: 0.014574
Train Epoch: Model Number: 1 0 [51200/60000 (85%)]	Loss: 0.017474
Train Epoch: Model Number: 1 10 [12800/60000 (21%)]	Loss: 0.023645
Train Epoch: Model Number: 1 10 [25600/60000 (43%)]	Loss: 0.023895
Train Epoch: Model Number: 1 10 [38400/60000 (64%)]	Loss: 0.014529
Train Epoch: Model Number: 1 10 [51200/60000 (85%)]	Loss: 0.022531
Train Epoch: Model Number: 1 20 [12800/60000 (21%)]	Loss: 0.017691
Train Epoch: Model Number: 1 20 [25600/60000 (43%)]	Loss: 0.015025
Train Epoch: Model Number: 1 20 [38400/60000 (64%)]	Loss: 0.014164
Train Epoch: Model Number: 1 20 [51200/60000 (85%)]	Loss: 0.021518
Train Epoch: Model Number: 1 30 [12800/60000 (21%)]	Loss: 0.015692
Train Epoch: Model Number: 1 30 [25600/60000 (43%)]	Loss: 0.014568
Train Epoch: Model Number: 1 30 [38400/60000 (64%)]	Loss: 0.012757

In [29]:
num_of_models = 3
models_adp, gamma_exp_adp = ADPGradient(initial_model, num_of_models)

Train Epoch: Model Number: 1 0 [51200/60000 (85%)]	Loss: -4.435918
Train Epoch: Model Number: 1 1 [51200/60000 (85%)]	Loss: -4.439062
Train Epoch: Model Number: 1 2 [51200/60000 (85%)]	Loss: -4.439041
Train Epoch: Model Number: 1 3 [51200/60000 (85%)]	Loss: -4.441585
Train Epoch: Model Number: 1 4 [51200/60000 (85%)]	Loss: -4.442135
Train Epoch: Model Number: 1 5 [51200/60000 (85%)]	Loss: -4.436696
Train Epoch: Model Number: 1 6 [51200/60000 (85%)]	Loss: -4.443707
Train Epoch: Model Number: 1 7 [51200/60000 (85%)]	Loss: -4.437622
Train Epoch: Model Number: 1 8 [51200/60000 (85%)]	Loss: -4.440589
Train Epoch: Model Number: 1 9 [51200/60000 (85%)]	Loss: -4.440548
Train Epoch: Model Number: 1 10 [51200/60000 (85%)]	Loss: -4.440617
Train Epoch: Model Number: 1 11 [51200/60000 (85%)]	Loss: -4.435962
Train Epoch: Model Number: 1 12 [51200/60000 (85%)]	Loss: -4.443596
Train Epoch: Model Number: 1 13 [51200/60000 (85%)]	Loss: -4.443573
Train Epoch: Model Number: 1 14 [51200/60000 (85%)]	Loss: 

## Save or Load submodels and optimized gamma

In [21]:
# Save all the models trained
for i in range(num_of_models):
    model = models[i]
    torch.save(model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(i) + '.pth')

In [32]:
# Save all the models trained
for i in range(num_of_models):
    model = models_adp[i]
    torch.save(model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model_adp' + str(i) + '.pth')

In [22]:
# Save the optimized gamma
torch.save(gamma_exp, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')

In [33]:
# Save the optimized gamma
torch.save(gamma_exp_adp, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp_adp.txt')

In [11]:
# Load the models from the local files
num_of_models = 3
models = []
for x in range(num_of_models):
    globals()['model%s' % x] = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(1) + '.pth')
    models.append(globals()['model%s' % x])
print(models)

[NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
), NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
), NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)]


In [ ]:
# Load the models from the local files
num_of_models = 3
models_adp = []
for x in range(num_of_models):
    globals()['model_adp%s' % x] = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model_adp' + str(1) + '.pth')
    models_adp.append(globals()['model_adp%s' % x])
print(models_adp)

In [12]:
# Load the optimized gamma from the local files
gamma_exp = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')
print(gamma_exp)

tensor([ 0.1407,  0.0500, -0.0431], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


In [ ]:
# Load the optimized gamma from the local files
gamma_exp_adp = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp_adp.txt')
print(gamma_exp)

## Ensemble Model Accuracy

In [17]:
#Find the accuracy of the ensemble model
# initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

loss = 0
correct = 0
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = initial_model(data)
        for i in range(num_of_models):
            model = models[i]
            if torch.cuda.is_available():
                model = model.cuda()
                output = output.cuda()
                gamma_temp = gamma_exp[i]
                gamma_temp = gamma_temp.cuda()
            output = output + gamma_temp * model(data)
        loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
loss /= len(train_loader.dataset)  
print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

<ipython-input-17-941e8f0dbfbb>:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Average loss: 1.5054, Accuracy: 57169/60000 (95.282%)



In [30]:
#Find the accuracy of the ensemble model
# initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

loss = 0
correct = 0
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = initial_model(data)
        for i in range(num_of_models):
            model = models_adp[i]
            if torch.cuda.is_available():
                model = model.cuda()
                output = output.cuda()
                gamma_temp = gamma_exp_adp[i]
                gamma_temp = gamma_temp.cuda()
            output = output + gamma_temp * model(data)
        loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
loss /= len(train_loader.dataset)  
print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

<ipython-input-30-694060482a16>:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)



Average loss: 1.5083, Accuracy: 57166/60000 (95.277%)



## CW Attack on Intial Model, Testing Robust Accuracy

In [14]:
# Attack the initial model using CW attack
initial_model.eval()

correct = 0
total = 0

cw_attack = CW(initial_model, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 42.52 %


## Creating Ensemble CW Attack

In [18]:
# Create ensemble CW attack
class Attack_ensemble(object):
    r"""
    Base class for all attacks.
    .. note::
        It automatically set device to the device where given model is.
        It temporarily changes the original model's training mode to `test`
        by `.eval()` only during an attack process.
    """
    def __init__(self, name, model, models, gamma):
        r"""
        Initializes internal attack state.
        Arguments:
            name (str) : name of an attack.
            model (torch.nn.Module): model to attack.
        """

        self.attack = name
        self.model = model
        self.models = models
        self.gamma = gamma
        self.model_name = str(model).split("(")[0]

        self.training = model.training
        self.device = next(model.parameters()).device
        
        self._targeted = 1
        self._attack_mode = 'original'
        self._return_type = 'float'

    def forward(self, *input):
        r"""
        It defines the computation performed at every call.
        Should be overridden by all subclasses.
        """
        raise NotImplementedError
        
    def set_attack_mode(self, mode):
        r"""
        Set the attack mode.
  
        Arguments:
            mode (str) : 'original' (DEFAULT)
                         'targeted' - Use input labels as targeted labels.
                         'least_likely' - Use least likely labels as targeted labels.
        """
        if self._attack_mode is 'only_original':
            raise ValueError("Changing attack mode is not supported in this attack method.")
            
        if mode=="original":
            self._attack_mode = "original"
            self._targeted = 1
            self._transform_label = self._get_label
        elif mode=="targeted":
            self._attack_mode = "targeted"
            self._targeted = -1
            self._transform_label = self._get_label
        elif mode=="least_likely":
            self._attack_mode = "least_likely"
            self._targeted = -1
            self._transform_label = self._get_least_likely_label
        else:
            raise ValueError(mode + " is not a valid mode. [Options : original, targeted, least_likely]")
            
    def set_return_type(self, type):
        r"""
        Set the return type of adversarial images: `int` or `float`.
        Arguments:
            type (str) : 'float' or 'int'. (DEFAULT : 'float')
        """
        if type == 'float':
            self._return_type = 'float'
        elif type == 'int':
            self._return_type = 'int'
        else:
            raise ValueError(type + " is not a valid type. [Options : float, int]")

    def save(self, save_path, data_loader, verbose=True):
        r"""
        Save adversarial images as torch.tensor from given torch.utils.data.DataLoader.
        Arguments:
            save_path (str) : save_path.
            data_loader (torch.utils.data.DataLoader) : data loader.
            verbose (bool) : True for displaying detailed information. (DEFAULT : True)
        """
        self.model.eval()

        image_list = []
        label_list = []

        correct = 0
        total = 0

        total_batch = len(data_loader)

        for step, (images, labels) in enumerate(data_loader):
            adv_images = self.__call__(images, labels)

            image_list.append(adv_images.cpu())
            label_list.append(labels.cpu())

            if self._return_type == 'int':
                adv_images = adv_images.float()/255

            if verbose:
                outputs = self.model(adv_images)
                for i in range(len(self.models)):
                    sub_model = self.models[i]
                    outputs = outputs + self.gamma[i] * sub_model(adv_images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(self.device)).sum()

                acc = 100 * float(correct) / total
                print('- Save Progress : %2.2f %% / Accuracy : %2.2f %%' % ((step+1)/total_batch*100, acc), end='\r')

        x = torch.cat(image_list, 0)
        y = torch.cat(label_list, 0)
        torch.save((x, y), save_path)
        print('\n- Save Complete!')

        self._switch_model()
        
    def _transform_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        """
        return labels
        
    def _get_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return input labels.
        """
        return labels
    
    def _get_least_likely_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return least likely labels.
        """
        outputs = self.model(images)
        for i in range(len(self.models)):
            sub_model = self.models[i]
            outputs = outputs + self.gamma[i] * sub_model(images)
        _, labels = torch.min(outputs.data, 1)
        labels = labels.detach_()
        return labels
    
    def _to_uint(self, images):
        r"""
        Function for changing the return type.
        Return images as int.
        """
        return (images*255).type(torch.uint8)

    def _switch_model(self):
        r"""
        Function for changing the training mode of the model.
        """
        if self.training:
            self.model.train()
            for i in range(len(self.models)):
                self.models[i].train()
        else:
            self.model.eval()
            for i in range(len(self.models)):
                self.models[i].eval()

    def __str__(self):
        info = self.__dict__.copy()
        
        del_keys = ['model', 'attack']
        
        for key in info.keys():
            if key[0] == "_" :
                del_keys.append(key)
                
        for key in del_keys:
            del info[key]
        
        info['attack_mode'] = self._attack_mode
        if info['attack_mode'] == 'only_original' :
            info['attack_mode'] = 'original'
            
        info['return_type'] = self._return_type
        
        return self.attack + "(" + ', '.join('{}={}'.format(key, val) for key, val in info.items()) + ")"

    def __call__(self, *input, **kwargs):
        self.model.eval()
        for i in range(len(self.models)):
            self.models[i].eval()
        images = self.forward(*input, **kwargs)
        self._switch_model()

        if self._return_type == 'int':
            images = self._to_uint(images)

        return images

<>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-18-1bd8957a13a4>:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._attack_mode is 'only_original':


In [19]:
# Create Ensemble CW Attack
import warnings

class CW_Ensemble(Attack_ensemble):
    r"""
    CW in the paper 'Towards Evaluating the Robustness of Neural Networks'
    [https://arxiv.org/abs/1608.04644]
    Distance Measure : L2
        
    Arguments:
        model (nn.Module): model to attack.
        c (float): c in the paper. parameter for box-constraint. (DEFALUT : 1e-4)    
            :math:`minimize \Vert\frac{1}{2}(tanh(w)+1)-x\Vert^2_2+c\cdot f(\frac{1}{2}(tanh(w)+1))`    
        kappa (float): kappa (also written as 'confidence') in the paper. (DEFALUT : 0)
            :math:`f(x')=max(max\{Z(x')_i:i\neq t\} -Z(x')_t, - \kappa)`
        steps (int): number of steps. (DEFALUT : 1000)
        lr (float): learning rate of the Adam optimizer. (DEFALUT : 0.01)
        
    .. warning:: With default c, you can't easily get adversarial images. Set higher c like 1.
    
    Shape:
        - images: :math:`(N, C, H, W)` where `N = number of batches`, `C = number of channels`,        `H = height` and `W = width`. It must have a range [0, 1].
        - labels: :math:`(N)` where each value :math:`y_i` is :math:`0 \leq y_i \leq` `number of labels`.
        - output: :math:`(N, C, H, W)`.
          
    Examples::
        >>> attack = torchattacks.CW(model, targeted=False, c=1e-4, kappa=0, steps=1000, lr=0.01)
        >>> adv_images = attack(images, labels)
        
    .. note:: NOT IMPLEMENTED methods in the paper due to time consuming.
    
        (1) Binary search for c.
        
        (2) Choosing best L2 adversaries.
    """
    def __init__(self, model, models, gamma, c=1e-4, kappa=0, steps=1000, lr=0.01):
        super(CW_Ensemble, self).__init__("CW", model, models, gamma)
        self.c = c
        self.kappa = kappa
        self.steps = steps
        self.lr = lr

    def forward(self, images, labels):
        r"""
        Overridden.
        """
        images = images.to(self.device)
        labels = labels.to(self.device)
        labels = self._transform_label(images, labels)

        # f-function in the paper
        def f(x):
            outputs = self.model(x)
            for i in range(len(self.models)):
                sub_model = self.models[i]
                outputs = outputs + self.gamma[i] * sub_model(x)
            one_hot_labels = torch.eye(len(outputs[0]))[labels].to(self.device)

            i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
            j = torch.masked_select(outputs, one_hot_labels.bool())

            return torch.clamp(self._targeted*(j-i), min=-self.kappa)

        w = torch.zeros_like(images).to(self.device)
        w.detach_()
        w.requires_grad = True

        optimizer = optim.Adam([w], lr=self.lr)
        prev = 1e10

        for step in range(self.steps):

            a = 1/2*(nn.Tanh()(w) + 1)

            loss1 = nn.MSELoss(reduction='sum')(a, images)
            loss2 = torch.sum(self.c*f(a))

            cost = loss1 + loss2

            optimizer.zero_grad()
            cost.backward(retain_graph=True)
            optimizer.step()

            # Early Stop when loss does not converge.
            if step % (self.steps//10) == 0:
                if cost > prev:
                    warnings.warn("Early stopped because the loss is not converged.")
                    return (1/2*(nn.Tanh()(w) + 1)).detach()
                prev = cost

            # print('- CW Attack Progress : %2.2f %%        ' %((step+1)/self.steps*100), end='\r')

        adv_images = (1/2*(nn.Tanh()(w) + 1)).detach()

        return adv_images

## CW Attack Ensemble Model, Testing Robust Accuracy

In [20]:
#Attack the ensemble model
initial_model.eval()
for i in range(num_of_models):
    models[i].eval()

correct = 0
total = 0

cw_attack = CW_Ensemble(model = initial_model,models = models,gamma = gamma_exp, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        for i in range(num_of_models):
            sub_model = models[i]
            outputs = outputs + gamma_exp[i] * sub_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 44.53 %


In [31]:
#Attack the ensemble model
initial_model.eval()
for i in range(num_of_models):
    models_adp[i].eval()

correct = 0
total = 0

cw_attack = CW_Ensemble(model = initial_model,models = models_adp,gamma = gamma_exp_adp, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        for i in range(num_of_models):
            sub_model = models_adp[i]
            outputs = outputs + gamma_exp_adp[i] * sub_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 42.19 %
